#### Libraries

In [7]:
from sklearn.model_selection import cross_val_score
import pandas as pd
import xgboost as xgb
import catboost as ctb

from collections import defaultdict

In [8]:
pd.set_option('display.max_columns', 100)

#### Load Data

In [9]:
wd = '/Users/konstantinivanov/Documents/work/PIK/files-pik_digital_day'

In [10]:
train = pd.read_csv(wd + '/train.csv')
test = pd.read_csv(wd + '/test.csv')
flat = pd.read_csv(wd + '/flat.csv', encoding='cp1251')
price = pd.read_csv(wd + '/price.csv', encoding='cp1251')
status = pd.read_csv(wd + '/status.csv', encoding='cp1251')

#### Observe Data

In [11]:
def dscr(file):
    print(file.shape)
    print(file.info())
    return file.head(20)

In [19]:
train.head(40)

,id,bulk_id,spalen,date1,start_square,value,price,mean_sq,mean_fl,plan_s,plan_m,plan_l,vid_0,vid_1,vid_2,month,month_cnt,Класс объекта,Количество помещений,Огорожена территория,Площадь земельного участка,Входные группы,Детский сад,Школа,Поликлиника,ФОК,Спортивная площадка,Автомойка,Кладовые,Колясочные,Кондиционирование,Вентлияция,Лифт,Система мусоротведения,Видеонаблюдение,Подземная парковка,Двор без машин,Машиномест,Площадь пром. зоны в радиусе 500 м,Площадь зеленой зоны в радиусе 500 м,До Кремля,До ТТК(км),До Садового(км),До большой дороги на машине(км),До удобной авторазвязки на машине(км),До метро пешком(км),До промки(км),До парка(км),До парка пешком(км),Станций метро от кольца,Площадь двора,Курс,Cтавка по ипотеке,Вклады до 1 года,Вклады от 1 года до 3 лет,Вклады свыше 3 лет
0,0,FF3814A9-9F7D-E711-8530-00505688958B,1,2017-10-01,13109.9,1124.5,135633.913043,38.00,8,147,198,0,0,169,176,10,34,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.698052,10.04,5.74,5.94,6.03
1,1,FF3814A9-9F7D-E711-8530-00505688958B,2,2017-10-01,19898.4,1412.3,128492.419825,58.01,9,139,120,83,30,136,177,10,34,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.698052,10.04,5.74,5.94,6.03
2,2,FF3814A9-9F7D-E711-8530-00505688958B,0,2017-10-01,2100.0,345.0,158237.634409,22.58,10,78,15,0,0,73,20,10,34,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.698052,10.04,5.74,5.94,6.03
3,3,FF3814A9-9F7D-E711-8530-00505688958B,3,2017-10-01,9614.9,769.4,123250.000000,78.81,8,50,0,72,0,30,92,10,34,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.698052,10.04,5.74,5.94,6.03
4,4,FF3814A9-9F7D-E711-8530-00505688958B,1,2017-11-01,11947.2,526.7,139420.634921,37.93,9,140,175,0,0,145,170,11,35,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.926552,9.87,5.17,7.07,6.20
5,5,FF3814A9-9F7D-E711-8530-00505688958B,2,2017-11-01,18500.7,1385.7,132136.990596,58.00,9,135,104,79,28,132,159,11,35,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.926552,9.87,5.17,7.07,6.20
6,6,FF3814A9-9F7D-E711-8530-00505688958B,0,2017-11-01,1738.8,91.0,163102.631579,22.88,10,60,16,0,0,58,18,11,35,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.926552,9.87,5.17,7.07,6.20
7,7,FF3814A9-9F7D-E711-8530-00505688958B,3,2017-11-01,8980.4,490.4,130635.087719,78.78,8,47,0,67,0,30,84,11,35,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.926552,9.87,5.17,7.07,6.20
8,8,FF3814A9-9F7D-E711-8530-00505688958B,1,2017-12-01,11266.0,1209.5,145131.649832,37.93,9,131,166,0,0,133,164,12,36,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.573855,9.73,5.31,6.28,5.89
9,9,FF3814A9-9F7D-E711-8530-00505688958B,2,2017-12-01,17058.7,869.6,136454.915254,57.83,9,128,98,68,27,124,144,12,36,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.573855,9.73,5.31,6.28,5.89


In [12]:
train.describe()

,id,spalen,start_square,value,price,mean_sq,mean_fl,plan_s,plan_m,plan_l,vid_0,vid_1,vid_2,month,month_cnt,Количество помещений,Площадь земельного участка,Детский сад,Школа,Поликлиника,ФОК,Кондиционирование,Вентлияция,Лифт,Видеонаблюдение,Машиномест,Площадь пром. зоны в радиусе 500 м,Площадь зеленой зоны в радиусе 500 м,До Кремля,До ТТК(км),До Садового(км),До большой дороги на машине(км),До удобной авторазвязки на машине(км),До метро пешком(км),До промки(км),До парка(км),До парка пешком(км),Станций метро от кольца,Площадь двора,Курс,Cтавка по ипотеке,Вклады до 1 года,Вклады от 1 года до 3 лет,Вклады свыше 3 лет
count,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.0,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000,8716.000000
mean,4357.500000,1.747820,2334.251036,267.558478,114823.919804,56.643324,10.545434,13.752639,12.496902,16.564250,3.493460,11.544516,22.812529,6.965581,27.187701,2838.982102,46.835568,515.428522,1346.437012,181.361863,0.129302,1.622992,0.633318,1.0,0.724071,4080.618747,54560.805760,102140.847637,20.312563,14.944000,17.870904,1.822969,6.685259,4.335750,0.501436,0.789666,1.132189,8.847751,6189.308398,60.612144,11.306884,6.258565,7.078382,7.131592
std,2516.236807,1.173904,3159.761082,376.172308,36301.007760,25.044095,5.072294,30.330175,28.673434,29.991525,11.454581,26.138621,39.809947,3.637026,7.968044,1405.837499,44.439546,280.050806,985.058320,258.189276,0.335554,0.930342,0.838686,0.0,0.514326,2988.453595,80182.395391,124208.337143,5.354376,5.181295,5.348252,1.695386,5.248174,5.042532,0.400826,0.512470,0.655344,3.608318,3303.846712,4.550858,1.118716,0.870123,0.786958,0.985009
min,0.000000,0.000000,19.300000,0.000000,30155.088235,18.900000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,7.000000,432.000000,1.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,80.000000,0.000000,0.000000,5.500000,0.550000,3.100000,0.060000,1.100000,0.250000,0.030000,0.050000,0.070000,2.000000,1365.000000,56.435614,9.730000,5.170000,5.940000,5.790000
25%,2178.750000,1.000000,274.485000,36.322500,87351.388889,37.660000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,22.000000,1931.000000,19.000000,220.000000,800.000000,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,997.000000,0.000000,12487.000000,18.000000,13.000000,16.000000,0.520000,3.500000,0.960000,0.170000,0.380000,0.650000,7.000000,4000.000000,57.744691,10.040000,5.620000,6.640000,6.050000
50%,4357.500000,2.000000,1161.785000,130.775000,102809.375000,54.560000,10.000000,1.000000,0.000000,2.000000,0.000000,0.000000,5.000000,7.000000,29.000000,2454.000000,23.250000,470.000000,1100.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.000000,3500.000000,13159.000000,31614.000000,20.000000,14.000000,17.000000,1.200000,5.400000,2.500000,0.390000,0.620000,1.000000,9.000000,5203.000000,58.926552,11.290000,6.170000,6.760000,7.000000
75%,6536.250000,3.000000,3119.060000,357.755000,138158.411017,76.985000,14.000000,15.000000,12.000000,21.000000,0.000000,10.000000,29.000000,10.000000,34.000000,3958.000000,70.000000,800.000000,2000.000000,500.000000,0.000000,3.000000,1.000000,1.0,1.000000,7050.000000,83805.000000,166632.000000,25.000000,19.000000,22.000000,2.700000,8.600000,5.600000,0.700000,1.200000,1.600000,12.000000,6822.000000,62.620005,12.210000,6.750000,7.390000,7.930000
max,8715.000000,4.000000,32494.300000,5746.580000,263000.000000,141.650000,32.000000,381.000000,336.000000,316.000000,141.000000,234.000000,376.000000,12.000000,37.000000,5579.000000,215.000000,1000.000000,3250.000000,600.000000,1.000000,3.

In [13]:
dscr(train)

(8716, 56)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8716 entries, 0 to 8715
Data columns (total 56 columns):
id                                       8716 non-null int64
bulk_id                                  8716 non-null object
spalen                                   8716 non-null int64
date1                                    8716 non-null object
start_square                             8716 non-null float64
value                                    8716 non-null float64
price                                    8716 non-null float64
mean_sq                                  8716 non-null float64
mean_fl                                  8716 non-null int64
plan_s                                   8716 non-null int64
plan_m                                   8716 non-null int64
plan_l                                   8716 non-null int64
vid_0                                    8716 non-null int64
vid_1                                    8716 non-null int64
vid_2             

,id,bulk_id,spalen,date1,start_square,value,price,mean_sq,mean_fl,plan_s,plan_m,plan_l,vid_0,vid_1,vid_2,month,month_cnt,Класс объекта,Количество помещений,Огорожена территория,Площадь земельного участка,Входные группы,Детский сад,Школа,Поликлиника,ФОК,Спортивная площадка,Автомойка,Кладовые,Колясочные,Кондиционирование,Вентлияция,Лифт,Система мусоротведения,Видеонаблюдение,Подземная парковка,Двор без машин,Машиномест,Площадь пром. зоны в радиусе 500 м,Площадь зеленой зоны в радиусе 500 м,До Кремля,До ТТК(км),До Садового(км),До большой дороги на машине(км),До удобной авторазвязки на машине(км),До метро пешком(км),До промки(км),До парка(км),До парка пешком(км),Станций метро от кольца,Площадь двора,Курс,Cтавка по ипотеке,Вклады до 1 года,Вклады от 1 года до 3 лет,Вклады свыше 3 лет
0,0,FF3814A9-9F7D-E711-8530-00505688958B,1,2017-10-01,13109.9,1124.5,135633.913043,38.00,8,147,198,0,0,169,176,10,34,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.698052,10.04,5.74,5.94,6.03
1,1,FF3814A9-9F7D-E711-8530-00505688958B,2,2017-10-01,19898.4,1412.3,128492.419825,58.01,9,139,120,83,30,136,177,10,34,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.698052,10.04,5.74,5.94,6.03
2,2,FF3814A9-9F7D-E711-8530-00505688958B,0,2017-10-01,2100.0,345.0,158237.634409,22.58,10,78,15,0,0,73,20,10,34,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.698052,10.04,5.74,5.94,6.03
3,3,FF3814A9-9F7D-E711-8530-00505688958B,3,2017-10-01,9614.9,769.4,123250.000000,78.81,8,50,0,72,0,30,92,10,34,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,57.698052,10.04,5.74,5.94,6.03
4,4,FF3814A9-9F7D-E711-8530-00505688958B,1,2017-11-01,11947.2,526.7,139420.634921,37.93,9,140,175,0,0,145,170,11,35,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.926552,9.87,5.17,7.07,6.20
5,5,FF3814A9-9F7D-E711-8530-00505688958B,2,2017-11-01,18500.7,1385.7,132136.990596,58.00,9,135,104,79,28,132,159,11,35,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.926552,9.87,5.17,7.07,6.20
6,6,FF3814A9-9F7D-E711-8530-00505688958B,0,2017-11-01,1738.8,91.0,163102.631579,22.88,10,60,16,0,0,58,18,11,35,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.926552,9.87,5.17,7.07,6.20
7,7,FF3814A9-9F7D-E711-8530-00505688958B,3,2017-11-01,8980.4,490.4,130635.087719,78.78,8,47,0,67,0,30,84,11,35,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.926552,9.87,5.17,7.07,6.20
8,8,FF3814A9-9F7D-E711-8530-00505688958B,1,2017-12-01,11266.0,1209.5,145131.649832,37.93,9,131,166,0,0,133,164,12,36,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.573855,9.73,5.31,6.28,5.89
9,9,FF3814A9-9F7D-E711-8530-00505688958B,2,2017-12-01,17058.7,869.6,136454.915254,57.83,9,128,98,68,27,124,144,12,36,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.40,0.28,0.58,0.93,8.0,7640.0,58.573855,9.73,5.31,6.28,5.89


In [18]:
train[list(map(lambda x: train.columns[x], [5, 7, 18]))][:16]

,value,mean_sq,Количество помещений
0,1124.5,38.00,1983
1,1412.3,58.01,1983
2,345.0,22.58,1983
3,769.4,78.81,1983
4,526.7,37.93,1983
5,1385.7,58.00,1983
6,91.0,22.88,1983
7,490.4,78.78,1983
8,1209.5,37.93,1983
9,869.6,57.83,1983


In [8]:
dscr(test)

(1770, 48)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1770 entries, 0 to 1769
Data columns (total 48 columns):
id                                       1770 non-null int64
bulk_id                                  1770 non-null object
spalen                                   1770 non-null int64
date1                                    1770 non-null object
price                                    1770 non-null float64
mean_sq                                  1770 non-null float64
mean_fl                                  1770 non-null int64
month                                    1770 non-null int64
month_cnt                                1770 non-null int64
Класс объекта                            1770 non-null object
Количество помещений                     1770 non-null int64
Огорожена территория                     1770 non-null object
Площадь земельного участка               1770 non-null float64
Входные группы                           1770 non-null object
Детский сад      

,id,bulk_id,spalen,date1,price,mean_sq,mean_fl,month,month_cnt,Класс объекта,Количество помещений,Огорожена территория,Площадь земельного участка,Входные группы,Детский сад,Школа,Поликлиника,ФОК,Спортивная площадка,Автомойка,Кладовые,Колясочные,Кондиционирование,Вентлияция,Лифт,Система мусоротведения,Видеонаблюдение,Подземная парковка,Двор без машин,Машиномест,Площадь пром. зоны в радиусе 500 м,Площадь зеленой зоны в радиусе 500 м,До Кремля,До ТТК(км),До Садового(км),До большой дороги на машине(км),До удобной авторазвязки на машине(км),До метро пешком(км),До промки(км),До парка(км),До парка пешком(км),Станций метро от кольца,Площадь двора,Курс,Cтавка по ипотеке,Вклады до 1 года,Вклады от 1 года до 3 лет,Вклады свыше 3 лет
0,0,FF3814A9-9F7D-E711-8530-00505688958B,1,2018-02-01,144734.016393,37.97,9,2,38,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.4,0.28,0.58,0.93,8.0,7640.0,56.806705,9.78,5.48,6.2,6.16
1,1,FF3814A9-9F7D-E711-8530-00505688958B,2,2018-02-01,136729.844961,57.78,9,2,38,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.4,0.28,0.58,0.93,8.0,7640.0,56.806705,9.78,5.48,6.2,6.16
2,2,FF3814A9-9F7D-E711-8530-00505688958B,0,2018-02-01,167787.719298,23.21,10,2,38,эконом,1983,нет,17.0,да,345,825,0,0,да,нет,да,да,1,1,1,контейнеры,0,да,да,1800,65357.0,25533.0,18.0,12.0,16.0,0.55,1.7,1.4,0.28,0.58,0.93,8.0,7640.0,56.806705,9.78,5.48,6.2,6.16


In [6]:
flat.describe()

,floor,spalen,stage_number,square,balcon,section,Количество помещений,Площадь земельного участка,Детский сад,Школа,Поликлиника,ФОК,Кондиционирование,Вентлияция,Лифт,Видеонаблюдение,Машиномест,Площадь пром. зоны в радиусе 500 м,Площадь зеленой зоны в радиусе 500 м,До Кремля,До ТТК(км),До Садового(км),До большой дороги на машине(км),До удобной авторазвязки на машине(км),До метро пешком(км),До промки(км),До парка(км),До парка пешком(км),Станций метро от кольца,Площадь двора
count,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.0,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000,111586.000000
mean,10.549818,1.635124,4.042980,52.779101,0.292017,3.561504,3285.426371,46.131309,603.730396,1608.794876,258.513344,0.104825,1.299858,0.654742,1.0,0.780636,3789.025630,54206.433630,120450.717079,21.023351,15.565955,18.504297,1.689106,6.750991,4.786389,0.444634,0.786290,1.118438,8.936623,6378.026975
std,6.331334,0.917628,2.978012,19.526824,0.454692,2.797704,1349.510777,38.321748,293.679865,984.741309,270.803072,0.306329,0.723768,0.880902,0.0,0.485875,2867.235594,80415.969443,131003.209512,5.483404,5.309929,5.550194,1.349062,5.470656,4.948639,0.322426,0.513333,0.682482,3.117576,2982.359856
min,1.000000,0.000000,1.000000,18.060000,0.000000,1.000000,432.000000,1.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,80.000000,0.000000,0.000000,5.500000,0.550000,3.100000,0.060000,1.100000,0.250000,0.030000,0.050000,0.070000,2.000000,1365.000000
25%,5.000000,1.000000,2.000000,38.270000,0.000000,1.000000,1983.000000,20.600000,410.000000,850.000000,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,1400.000000,0.000000,7000.000000,19.000000,14.000000,16.000000,0.660000,3.400000,1.300000,0.170000,0.360000,0.580000,7.000000,4359.000000
50%,10.000000,2.000000,3.000000,49.920000,0.000000,3.000000,3451.000000,35.000000,650.000000,1650.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.000000,2625.000000,15743.000000,60565.000000,20.000000,15.000000,17.000000,1.400000,5.600000,4.400000,0.380000,0.660000,1.000000,9.000000,5698.000000
75%,15.000000,2.000000,5.000000,65.300000,1.000000,5.000000,4146.000000,59.400000,820.000000,2500.000000,510.000000,0.000000,1.000000,2.000000,1.0,1.000000,6500.000000,74952.000000,226658.000000,25.000000,19.000000,22.000000,2.300000,7.200000,5.700000,0.600000,1.200000,1.600000,12.000000,7133.000000
max,33.000000,4.000000,30.000000,148.500000,1.000000,17.000000,5579.000000,215.000000,1000.000000,3250.000000,600.000000,1.000000,3.000000,2.000000,1.0,2.000000,9745.000000,378372.000000,516706.000000,38.000000,33.000000,36.000000,6.000000,26.000000,21.000000,1.500000,1.950000,2.700000,15.000000,16473.000000


In [9]:
dscr(flat)

(111586, 51)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111586 entries, 0 to 111585
Data columns (total 51 columns):
id_sec                                   111586 non-null object
floor                                    111586 non-null int64
spalen                                   111586 non-null float64
stage_number                             111586 non-null int64
square                                   111586 non-null float64
balcon                                   111586 non-null int64
otdelka                                  103215 non-null object
plan0                                    111586 non-null object
id_bulk                                  111586 non-null object
section                                  111586 non-null int64
date_settle                              103570 non-null object
date_salestart                           111586 non-null object
id_gk                                    111586 non-null object
id_flatwork                              11158

,id_sec,floor,spalen,stage_number,square,balcon,otdelka,plan0,id_bulk,section,date_settle,date_salestart,id_gk,id_flatwork,Класс объекта,Количество помещений,Огорожена территория,Площадь земельного участка,Входные группы,Детский сад,Школа,Поликлиника,ФОК,Спортивная площадка,Автомойка,Кладовые,Колясочные,Кондиционирование,Вентлияция,Лифт,Система мусоротведения,Видеонаблюдение,Подземная парковка,Двор без машин,Машиномест,Площадь пром. зоны в радиусе 500 м,Площадь зеленой зоны в радиусе 500 м,До Кремля,До ТТК(км),До Садового(км),До большой дороги на машине(км),До удобной авторазвязки на машине(км),До метро пешком(км),До промки(км),До парка(км),До парка пешком(км),Станций метро от кольца,Площадь двора,vid,sale,plan_size
0,846EA675-93FF-E411-8098-001EC9D56418,2,3.0,1,85.9,0,Не производится,3M-1*,4B3B6D6A-93FF-E411-8098-001EC9D56418,1,2017-03-15 21:00:00,2015-06-14 21:00:00,32671714-491B-E511-82D8-001EC9D5643C,8A11F41C-B3FF-E411-8098-001EC9D56418,комфорт,2284,нет,19.0,да,220,0,0,0,да,нет,да,да,3,1,1,контейнеры,1,нет,да,997,183441.0,0.0,18.0,13.0,16.0,1.2,3.9,0.96,0.06,0.61,1.1,10.0,14852.0,хороший,2016-07-22 21:00:00,L
1,846EA675-93FF-E411-8098-001EC9D56418,2,1.0,2,34.2,0,Не производится,1S-1*,4B3B6D6A-93FF-E411-8098-001EC9D56418,1,2017-03-15 21:00:00,2015-06-14 21:00:00,32671714-491B-E511-82D8-001EC9D5643C,9511F41C-B3FF-E411-8098-001EC9D56418,комфорт,2284,нет,19.0,да,220,0,0,0,да,нет,да,да,3,1,1,контейнеры,1,нет,да,997,183441.0,0.0,18.0,13.0,16.0,1.2,3.9,0.96,0.06,0.61,1.1,10.0,14852.0,хороший,2016-06-11 21:00:00,S
2,846EA675-93FF-E411-8098-001EC9D56418,2,1.0,3,33.7,0,Не производится,1S-2*,4B3B6D6A-93FF-E411-8098-001EC9D56418,1,2017-03-15 21:00:00,2015-06-14 21:00:00,32671714-491B-E511-82D8-001EC9D5643C,A011F41C-B3FF-E411-8098-001EC9D56418,комфорт,2284,нет,19.0,да,220,0,0,0,да,нет,да,да,3,1,1,контейнеры,1,нет,да,997,183441.0,0.0,18.0,13.0,16.0,1.2,3.9,0.96,0.06,0.61,1.1,10.0,14852.0,хороший,2015-08-29 21:00:00,S


In [10]:
dscr(price)

(647698, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647698 entries, 0 to 647697
Data columns (total 5 columns):
id_flatwork       647698 non-null object
datefrom          647698 non-null object
dateto            647698 non-null object
pricem2           647698 non-null float64
date_salestart    647698 non-null object
dtypes: float64(1), object(4)
memory usage: 24.7+ MB
None


,id_flatwork,datefrom,dateto,pricem2,date_salestart
0,E423E9CF-1416-E711-96FE-001EC9D5643C,2017-06-19 21:00:00,2017-12-29 21:00:00,90200.0,2017-04-18 21:00:00
1,EE9F1F73-01AF-E611-9FBE-001EC9D5643C,2017-02-13 21:00:00,2100-01-01 00:00:00,83000.0,2015-02-10 15:08:00
2,7DABC154-02AF-E611-9FBE-001EC9D5643C,2017-02-14 21:00:00,2017-11-01 21:00:00,106900.0,2016-04-22 08:17:00


In [11]:
dscr(status)

(696799, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696799 entries, 0 to 696798
Data columns (total 5 columns):
id_flatwork    696799 non-null object
datefrom       696799 non-null object
dateto         696799 non-null object
stat           696799 non-null int64
stat_name      696799 non-null object
dtypes: int64(1), object(4)
memory usage: 26.6+ MB
None


,id_flatwork,datefrom,dateto,stat,stat_name
0,6FF0141C-FDAE-E611-9FBE-001EC9D5643C,2017-01-31 21:00:00,2017-01-31 21:00:00,100000003,Зарезервирован под клиента
1,6E603D81-FEAE-E611-9FBE-001EC9D5643C,2017-01-31 21:00:00,2100-01-01 00:00:00,100000004,Реализован
2,0D0A77B7-7CE1-E611-B887-001EC9D5643C,2017-01-31 21:00:00,2100-01-01 00:00:00,100000004,Реализован


#### Work with Features 

In [12]:
feat_cols = list(test.columns[1:])
targ_cols = train.columns[5]

In [23]:
nam_cat_features = feat_cols[0:3] + feat_cols[8:9] + feat_cols[10:11] + feat_cols[12:13] + feat_cols[17:28]

In [24]:
nam_cat_features

['bulk_id',
 'spalen',
 'date1',
 'Класс объекта',
 'Огорожена территория',
 'Входные группы',
 'Спортивная площадка',
 'Автомойка',
 'Кладовые',
 'Колясочные',
 'Кондиционирование',
 'Вентлияция',
 'Лифт',
 'Система мусоротведения',
 'Видеонаблюдение',
 'Подземная парковка',
 'Двор без машин']

In [25]:
cat_features = []
for itr, nam in enumerate(feat_cols):
    if nam in nam_cat_features:
        cat_features.append(itr)

In [26]:
cat_features

[0, 1, 2, 8, 10, 12, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

In [13]:
X_train = train[feat_cols].values
y_train = train[targ_cols].values

X_test = test[feat_cols].values

#### CatBoost

In [16]:
ctb_model = ctb.CatBoostRegressor(loss_function='RMSE', random_seed=42)

In [27]:
ctb_model.fit(X=X_train, y=y_train, cat_features=cat_features)

0:	learn: 455.5563969	total: 97.2ms	remaining: 1m 37s
1:	learn: 449.4861459	total: 124ms	remaining: 1m 1s
2:	learn: 444.1750516	total: 147ms	remaining: 48.7s
3:	learn: 438.6110962	total: 171ms	remaining: 42.6s
4:	learn: 433.6171999	total: 196ms	remaining: 38.9s
5:	learn: 428.4889673	total: 219ms	remaining: 36.3s
6:	learn: 424.0153786	total: 243ms	remaining: 34.4s
7:	learn: 419.8656533	total: 272ms	remaining: 33.7s
8:	learn: 415.2799895	total: 298ms	remaining: 32.8s
9:	learn: 411.2944238	total: 332ms	remaining: 32.8s
10:	learn: 407.4494342	total: 356ms	remaining: 32s
11:	learn: 403.5704761	total: 383ms	remaining: 31.5s
12:	learn: 399.7133087	total: 407ms	remaining: 30.9s
13:	learn: 395.8850512	total: 431ms	remaining: 30.4s
14:	learn: 393.0148295	total: 444ms	remaining: 29.2s
15:	learn: 389.7190956	total: 472ms	remaining: 29.1s
16:	learn: 386.5430191	total: 501ms	remaining: 29s
17:	learn: 383.6846720	total: 536ms	remaining: 29.3s
18:	learn: 380.8475211	total: 564ms	remaining: 29.1s
19:	l

161:	learn: 277.5977480	total: 4.87s	remaining: 25.2s
162:	learn: 277.4424357	total: 4.9s	remaining: 25.2s
163:	learn: 277.0890101	total: 4.93s	remaining: 25.1s
164:	learn: 277.0885393	total: 4.93s	remaining: 24.9s
165:	learn: 276.9503388	total: 4.96s	remaining: 24.9s
166:	learn: 276.7365742	total: 4.99s	remaining: 24.9s
167:	learn: 276.5706174	total: 5.01s	remaining: 24.8s
168:	learn: 276.4435243	total: 5.04s	remaining: 24.8s
169:	learn: 276.2992927	total: 5.07s	remaining: 24.8s
170:	learn: 276.0902432	total: 5.09s	remaining: 24.7s
171:	learn: 276.0010720	total: 5.12s	remaining: 24.6s
172:	learn: 275.7416667	total: 5.14s	remaining: 24.6s
173:	learn: 275.5261618	total: 5.17s	remaining: 24.5s
174:	learn: 275.4021392	total: 5.18s	remaining: 24.4s
175:	learn: 275.2742906	total: 5.21s	remaining: 24.4s
176:	learn: 274.8614361	total: 5.23s	remaining: 24.3s
177:	learn: 274.6354960	total: 5.26s	remaining: 24.3s
178:	learn: 274.4055637	total: 5.28s	remaining: 24.2s
179:	learn: 273.9364156	total

317:	learn: 251.7996528	total: 9.69s	remaining: 20.8s
318:	learn: 251.6828698	total: 9.71s	remaining: 20.7s
319:	learn: 251.5567656	total: 9.74s	remaining: 20.7s
320:	learn: 251.3447022	total: 9.77s	remaining: 20.7s
321:	learn: 251.1078461	total: 9.8s	remaining: 20.6s
322:	learn: 250.9778494	total: 9.84s	remaining: 20.6s
323:	learn: 250.7899039	total: 9.86s	remaining: 20.6s
324:	learn: 250.6897238	total: 9.9s	remaining: 20.6s
325:	learn: 250.5341360	total: 9.94s	remaining: 20.5s
326:	learn: 250.4490203	total: 9.97s	remaining: 20.5s
327:	learn: 250.2873721	total: 10s	remaining: 20.5s
328:	learn: 250.2390816	total: 10s	remaining: 20.5s
329:	learn: 249.9972477	total: 10.1s	remaining: 20.4s
330:	learn: 249.8629461	total: 10.1s	remaining: 20.4s
331:	learn: 249.8324428	total: 10.1s	remaining: 20.4s
332:	learn: 249.6789220	total: 10.2s	remaining: 20.3s
333:	learn: 249.5611049	total: 10.2s	remaining: 20.3s
334:	learn: 249.4040546	total: 10.2s	remaining: 20.3s
335:	learn: 249.3203120	total: 10.

474:	learn: 235.2403786	total: 14.6s	remaining: 16.1s
475:	learn: 235.1312188	total: 14.6s	remaining: 16.1s
476:	learn: 235.0837646	total: 14.6s	remaining: 16.1s
477:	learn: 235.0693906	total: 14.7s	remaining: 16s
478:	learn: 235.0167709	total: 14.7s	remaining: 16s
479:	learn: 234.9694909	total: 14.7s	remaining: 15.9s
480:	learn: 234.9006667	total: 14.7s	remaining: 15.9s
481:	learn: 234.8233126	total: 14.8s	remaining: 15.9s
482:	learn: 234.7387209	total: 14.8s	remaining: 15.8s
483:	learn: 234.6481890	total: 14.8s	remaining: 15.8s
484:	learn: 234.5389787	total: 14.8s	remaining: 15.8s
485:	learn: 234.5160306	total: 14.9s	remaining: 15.7s
486:	learn: 234.4433293	total: 14.9s	remaining: 15.7s
487:	learn: 234.4253838	total: 14.9s	remaining: 15.7s
488:	learn: 234.3882236	total: 15s	remaining: 15.6s
489:	learn: 234.3082463	total: 15s	remaining: 15.6s
490:	learn: 234.2877174	total: 15s	remaining: 15.6s
491:	learn: 234.1517930	total: 15.1s	remaining: 15.5s
492:	learn: 234.1112761	total: 15.1s	r

628:	learn: 225.3657079	total: 19.1s	remaining: 11.3s
629:	learn: 225.2900478	total: 19.1s	remaining: 11.2s
630:	learn: 225.2879051	total: 19.2s	remaining: 11.2s
631:	learn: 225.2541143	total: 19.2s	remaining: 11.2s
632:	learn: 225.1838559	total: 19.2s	remaining: 11.1s
633:	learn: 225.1068871	total: 19.3s	remaining: 11.1s
634:	learn: 225.0630169	total: 19.3s	remaining: 11.1s
635:	learn: 225.0517857	total: 19.3s	remaining: 11.1s
636:	learn: 225.0367895	total: 19.4s	remaining: 11s
637:	learn: 224.9988687	total: 19.4s	remaining: 11s
638:	learn: 224.9261898	total: 19.4s	remaining: 11s
639:	learn: 224.8419901	total: 19.4s	remaining: 10.9s
640:	learn: 224.7365309	total: 19.5s	remaining: 10.9s
641:	learn: 224.6390803	total: 19.5s	remaining: 10.9s
642:	learn: 224.6079481	total: 19.5s	remaining: 10.8s
643:	learn: 224.5634238	total: 19.6s	remaining: 10.8s
644:	learn: 224.5266151	total: 19.6s	remaining: 10.8s
645:	learn: 224.4497774	total: 19.6s	remaining: 10.8s
646:	learn: 224.3984183	total: 19.

781:	learn: 217.8217569	total: 23.9s	remaining: 6.67s
782:	learn: 217.7682319	total: 24s	remaining: 6.64s
783:	learn: 217.6794489	total: 24s	remaining: 6.61s
784:	learn: 217.5997943	total: 24s	remaining: 6.58s
785:	learn: 217.5162774	total: 24.1s	remaining: 6.55s
786:	learn: 217.4775003	total: 24.1s	remaining: 6.52s
787:	learn: 217.4329490	total: 24.1s	remaining: 6.49s
788:	learn: 217.3655547	total: 24.1s	remaining: 6.46s
789:	learn: 217.3247846	total: 24.2s	remaining: 6.43s
790:	learn: 217.3055746	total: 24.2s	remaining: 6.39s
791:	learn: 217.2723638	total: 24.2s	remaining: 6.36s
792:	learn: 217.2574421	total: 24.3s	remaining: 6.33s
793:	learn: 217.2112641	total: 24.3s	remaining: 6.3s
794:	learn: 217.1583454	total: 24.3s	remaining: 6.27s
795:	learn: 217.1319466	total: 24.3s	remaining: 6.24s
796:	learn: 217.0745452	total: 24.4s	remaining: 6.21s
797:	learn: 217.0526705	total: 24.4s	remaining: 6.17s
798:	learn: 217.0294627	total: 24.4s	remaining: 6.14s
799:	learn: 217.0014579	total: 24.5

936:	learn: 211.4546535	total: 28.9s	remaining: 1.94s
937:	learn: 211.4219634	total: 28.9s	remaining: 1.91s
938:	learn: 211.3643390	total: 28.9s	remaining: 1.88s
939:	learn: 211.3156644	total: 29s	remaining: 1.85s
940:	learn: 211.2736898	total: 29s	remaining: 1.82s
941:	learn: 211.2412822	total: 29s	remaining: 1.79s
942:	learn: 211.2251560	total: 29.1s	remaining: 1.76s
943:	learn: 211.1870058	total: 29.1s	remaining: 1.73s
944:	learn: 211.1660108	total: 29.1s	remaining: 1.7s
945:	learn: 211.1144216	total: 29.2s	remaining: 1.66s
946:	learn: 211.0753842	total: 29.2s	remaining: 1.63s
947:	learn: 211.0129298	total: 29.2s	remaining: 1.6s
948:	learn: 210.9800064	total: 29.3s	remaining: 1.57s
949:	learn: 210.9366338	total: 29.3s	remaining: 1.54s
950:	learn: 210.9234840	total: 29.3s	remaining: 1.51s
951:	learn: 210.8711313	total: 29.3s	remaining: 1.48s
952:	learn: 210.8569995	total: 29.4s	remaining: 1.45s
953:	learn: 210.8432291	total: 29.4s	remaining: 1.42s
954:	learn: 210.8240105	total: 29.4s

In [28]:
y_test = ctb_model.predict(X_test)

In [30]:
len(y_test)

1770

#### Writing Answer

In [58]:
def write_ans(fl_name):
    dict_ans = defaultdict(list)
    dict_ans['id'] = list(range(len(y_test)))
    dict_ans['value'] = list(y_test)
    df_ans = pd.DataFrame.from_dict(dict_ans)
    df_ans.to_csv(wd+'/%s'%fl_name, index=False)
    return 'Answer succesfully writed'

In [59]:
write_ans('ctb_just.csv')

'Answer succesfully writed'